# 1. rubot Mecanum model description

Diferent models could be obtained:
- Commercial robot nexus robot
- Costomized nexus robot
- Custom designed model 

For a custom model created in SolidWorks, there is an add-in that allows the conversion of SW Parts and Assemblies into a URDF file. The exporter will create a ROS-like package that contains a directory for meshes, textures and robots (urdf files).: http://wiki.ros.org/sw_urdf_exporter




## 1.1. Commercial nexus robot

This commercial robot can be found in: https://www.nexusrobot.com/product/4wd-mecanum-wheel-mobile-arduino-robotics-car-10011.html

There is already a URDF model extracted from: https://github.com/RBinsonB/nexus_4wd_mecanum_simulator

Run the simulation for one nexus robot by using the following command line:

In [ ]:
roslaunch nexus_4wd_mecanum_gazebo nexus_4wd_mecanum_world.launch

![Getting Started](./Images/1_rubot_gazebo1.png)

To launch two nexus robots, use the following launch file:

In [ ]:
roslaunch nexus_4wd_mecanum_gazebo nexus_4wd_mecanum_multi.launch

![Getting Started](./Images/1_rubot_gazebo2.png)

##1.2. Costomized nexus robot

We have added:
- Camera sensor
- LIDAR sensor
- Kinematic driver for Holonomic movement


### Camera sensor

This sensor is integrated as a link and fixed joint for visual purposes:

In [ ]:
  <!-- 2D Camera as a mesh of actual PiCamera -->
  <link name="camera">
    <visual>
      <origin rpy="0 1.570795 0" xyz="0 0 0"/>
      <geometry>
        <mesh filename="package://nexus_4wd_mecanum_description/meshes/piCamera.stl" scale="0.0025 0.0025 0.0025"/>
      </geometry>
      <material name="yellow"/>
    </visual>
    <collision>
      <origin rpy="0 1.570795 0" xyz="0 0 0"/>
      <geometry>
        <box size="0.075 0.075 0.025"/>
      </geometry>
    </collision>
    <inertial>
      <origin rpy="0 1.570795 0" xyz="0 0 0"/>
      <mass value="1e-3"/>
      <inertia ixx="1e-6" ixy="0" ixz="0" iyy="1e-6" iyz="0" izz="1e-6"/>
    </inertial>
  </link>
  <!-- 2D Camera JOINT base_link -->
  <joint name="joint_camera" type="fixed">
    <axis xyz="0 0 1"/>
    <origin rpy="0 0 0" xyz="0.16 0 0.05"/>
    <parent link="base_link"/>
    <child link="camera"/>
  </joint>

A driver is needed to view the images.

In [ ]:
  <!-- 2D Camera controller -->
  <gazebo reference="camera">
    <sensor name="camera1" type="camera">
      <update_rate>30.0</update_rate>
      <camera name="front">
        <horizontal_fov>1.3962634</horizontal_fov>
        <image>
          <width>800</width>
          <height>800</height>
          <format>R8G8B8</format>
        </image>
        <clip>
          <near>0.02</near>
          <far>300</far>
        </clip>
      </camera>
      <plugin filename="libgazebo_ros_camera.so" name="camera_controller">
        <alwaysOn>true</alwaysOn>
        <updateRate>0.0</updateRate>
        <cameraName>rubot/camera1</cameraName>
        <imageTopicName>image_raw</imageTopicName>
        <cameraInfoTopicName>camera_info</cameraInfoTopicName>
        <frameName>camera</frameName>
        <hackBaseline>0.07</hackBaseline>
        <distortionK1>0.0</distortionK1>
        <distortionK2>0.0</distortionK2>
        <distortionK3>0.0</distortionK3>
        <distortionT1>0.0</distortionT1>
        <distortionT2>0.0</distortionT2>
      </plugin>
    </sensor>
  </gazebo>

### LIDAR sensor

This sensor is integrated as a link and fixed joint for visual purposes:

In [ ]:
  <!-- LIDAR base_scan -->
  <link name="base_scan">
    <visual name="sensor_body">
      <origin rpy="0 0 0" xyz="0 0 0.04"/>
      <geometry>
        <mesh filename="package://nexus_4wd_mecanum_description/meshes/X4.stl" scale="0.0015 0.0015 0.0015"/>
      </geometry>
      <material name="yellow"/>
    </visual>
    <collision>
      <origin rpy="0 0 0" xyz="0 0 0.04"/>
      <geometry>
        <cylinder length="0.01575" radius="0.0275"/>
      </geometry>
    </collision>
    <inertial>
      <origin rpy="0 0 0" xyz="0 0 0.4"/>
      <mass value="0.057"/>
      <inertia ixx="0.001" ixy="0.0" ixz="0.0" iyy="0.001" iyz="0.0" izz="0.001"/>
    </inertial>
  </link>
  <!-- LIDAR base_scan JOINT base_link -->
  <joint name="scan_joint" type="fixed">
    <axis xyz="0 0 1"/>
    <origin rpy="0 0 0" xyz="0 0 0.09"/>
    <parent link="base_link"/>
    <child link="base_scan"/>
  </joint>

A driver is needed to see the 720 laser distance points:

In [ ]:
  <!-- Laser Distance Sensor YDLIDAR X4 controller-->
  <gazebo reference="base_scan">
    <sensor name="lds_lfcd_sensor" type="ray">
      <pose>0 0 0 0 0 0</pose>
      <visualize>false</visualize>
      <update_rate>5</update_rate>
      <ray>
        <scan>
          <horizontal>
            <samples>720</samples>
            <resolution>0.5</resolution>
            <min_angle>0.0</min_angle>
            <max_angle>6.28319</max_angle>
          </horizontal>
        </scan>
        <range>
          <min>0.12</min>
          <max>10</max>
          <resolution>0.015</resolution>
        </range>
        <noise>
          <type>gaussian</type>
          <!-- Noise parameters based on published spec for YDLIDAR X4
              is 1.5% at half range 4m (= 60mm, "+-160mm" accuracy at max. range 8m).
              A mean of 0.0m and stddev of 0.020m will put 99.7% of samples
              within 0.16m of the true reading. -->
          <mean>0.0</mean>
          <stddev>0.02</stddev>
        </noise>
      </ray>
      <plugin filename="libgazebo_ros_laser.so" name="gazebo_ros_lds_lfcd_controller">
        <!-- topicName>/gopigo/scan</topicName -->
        <topicName>scan</topicName>
        <frameName>base_scan</frameName>
      </plugin>
    </sensor>
  </gazebo>

A driver is needed to describe the kinematics.This kinematics is described in the "libgazebo_ros_control.so" file and the URDF model will contain the specufic gazebo plugin:

In [ ]:
<?xml version="1.0"?>
<robot>
  <!-- Mecanum drive controller -->
  <gazebo>
    <!--<plugin name="gazebo_ros_control" filename="libgazebo_ros_control.so">
      <robotNamespace>/</robotNamespace>
    </plugin>-->

    <plugin name="nexus_ros_force_based_move" filename="libnexus_ros_force_based_move.so">
      <commandTopic>cmd_vel</commandTopic>
      <odometryTopic>odom</odometryTopic>
      <odometryFrame>odom</odometryFrame>
      <odometryRate>25.0</odometryRate>
      <robotBaseFrame>base_footprint</robotBaseFrame>
      <cmdVelTimeOut>0.25</cmdVelTimeOut>
      <publishOdometryTf>0</publishOdometryTf>
      <yaw_velocity_p_gain>1.0</yaw_velocity_p_gain>
      <x_velocity_p_gain>15.0</x_velocity_p_gain>
      <y_velocity_p_gain>15.0</y_velocity_p_gain>
      <max_x_velocity>0.6</max_x_velocity>
      <max_y_velocity>0.6</max_y_velocity>
      <max_yaw_velocity>0.5</max_yaw_velocity>
    </plugin>
  </gazebo>

To control the POSE with cmd_vel we can install the package: 
- For Holonomic movements: http://wiki.ros.org/teleop_twist_keyboard
- for non-holonomic movements: http://wiki.ros.org/teleop_tools

In [ ]:
sudo apt-get install ros-melodic-teleop-tools
or
sudo apt-get install ros-melodic-teleop-twist-keyboard

To launch the control:

In [ ]:
rosrun key_teleop key_teleop.py /key_vel:=/cmd_vel
or
rosrun teleop_twist_keyboard teleop_twist_keyboard.py

# rUBot mecanum model
This robot includes a:
- 2D camera
- LIDAR

we have added the corresponding links and joints in "rubot_mecanum.xacro" file and the gazebo plugins in "rubot_mecanum.gazebo" file

In [ ]:
roslaunch nexus_4wd_mecanum_gazebo rubot_mecanum_world.launch

In [ ]:
roslaunch nexus_4wd_mecanum_description rubot_mecanum_rviz.launch

<img src="./Images/1_rubot_mecanum2.png">

### Plugin for driving control
We can choose:
- the one designed for Nexus_mecanum robot: libnexus_ros_force_based_move.so
- a generic one for holonomic movements: libgazebo_ros_planar_move.so

#### Nexus driving controller:
This driver is based on a force controlled driver and is custom designed in specific cpp file "nexus_ros_force_based_move.cpp" in "nexus_4wd_mecanum_gazebo" package.

Carefull!

Be sure in gazebo plugin, the /odom topic will be published:<publishOdometryTf>true</publishOdometryTf>

In [ ]:
  <!-- Mecanum drive controller -->
  <gazebo>
    <!--<plugin name="gazebo_ros_control" filename="libgazebo_ros_control.so">
      <robotNamespace>/</robotNamespace>
    </plugin>-->

    <plugin name="nexus_ros_force_based_move" filename="libnexus_ros_force_based_move.so">
      <commandTopic>cmd_vel</commandTopic>
      <odometryTopic>odom</odometryTopic>
      <odometryFrame>odom</odometryFrame>
      <odometryRate>25.0</odometryRate>
      <robotBaseFrame>base_footprint</robotBaseFrame>
      <cmdVelTimeOut>0.25</cmdVelTimeOut>
      <publishOdometryTf>true</publishOdometryTf>
      <yaw_velocity_p_gain>1.0</yaw_velocity_p_gain>
      <x_velocity_p_gain>15.0</x_velocity_p_gain>
      <y_velocity_p_gain>15.0</y_velocity_p_gain>
      <max_x_velocity>0.6</max_x_velocity>
      <max_y_velocity>0.6</max_y_velocity>
      <max_yaw_velocity>0.5</max_yaw_velocity>
    </plugin>
  </gazebo>

#### ROS holonomic driving controller:
This driver is the "Planar Move Plugin" and is described in Gazebo tutorials: http://gazebosim.org/tutorials?tut=ros_gzplugins#AddingaModelPlugin


In [ ]:
 <gazebo>
    <plugin name="Mecanum_controller" filename="libgazebo_ros_planar_move.so">
      <commandTopic>cmd_vel</commandTopic>
      <odometryTopic>odom</odometryTopic>
      <odometryFrame>odom</odometryFrame>
      <odometryRate>50.0</odometryRate>
      <robotBaseFrame>base_footprint</robotBaseFrame>
    </plugin>
  </gazebo>

### Nexus robot control
You can test the robot model in Gazebo and rviz to verufy the driving and sensor functionalities:

In [ ]:
rosrun teleop_twist_keyboard teleop_twist_keyboard.py 